In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F
import os

In [2]:
spark = SparkSession.builder \
    .appName("pytest_pyspark") \
    .enableHiveSupport() \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/01/28 17:18:16 WARN Utils: Your hostname, DESKTOP-RPDLSP7 resolves to a loopback address: 127.0.1.1; using 172.17.212.36 instead (on interface eth0)
24/01/28 17:18:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/28 17:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.option("header",True).option("inferSchema",True).csv(os.getcwd() + "/tests/data/input/passenger_dataset.csv")
df.printSchema()

root
 |-- airline: string (nullable = true)
 |-- origin_city: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- passenger_name: string (nullable = true)
 |-- passenger_id: integer (nullable = true)



In [5]:
# Test case 1: Passenger IDs should not be NULL
df.select("passenger_id").filter("passenger_id is null").count() == 0

False

In [9]:
# Test case 2: One passenger has a unique record for every airline route
df \
.select("passenger_id","airline") \
.filter("passenger_id is not null") \
.groupBy("airline","passenger_id") \
.count() \
.filter("count > 1") \
.count()

108

In [22]:
# Test case 3: Every origin city must have same destination city
route_map = {
    "Mumbai": "Dubai",
    "Singapore": "Tokyo",
    "Berlin": "New York"
}
for origin, dest in route_map.items():
    check_origin = df \
    .select("origin_city","destination_city") \
    .filter(f"origin_city == '{origin}'") \
    .select("destination_city") \
    .distinct() \
    .collect()[0][0] 
    print(check_origin == dest)


True
True
True


In [13]:
# Test case 4: List of 300 passengers matching for Airline A
input_df = (
    df
    .select("airline", "origin_city", "destination_city", "passenger_name")
    .filter("airline == 'Airline A'")
    .orderBy("passenger_name")
)
expected_df = (
    spark
    .read
    .option("header",True)
    .option("inferSchema",True)
    .csv(os.getcwd() + "/tests/data/expected/airline_a_passengers.csv")
    .orderBy("passenger_name")
)
diff_df = input_df.exceptAll(expected_df)
diff_df.count()

0